In [1]:
import pyexasol
import pandas as pd
import boto3
import time
import io
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

dwh_host = '***'
dwh_port = '8563'
dwh_user = '***'
dwh_pass = '***'

#connect to db
dwh_conn = pyexasol.connect(dsn=dwh_host + ':' + dwh_port, user=dwh_user, password=dwh_pass)


## set of functions to get different data sets

In [2]:

#get iterator month data
def get_iterator_data():

    #get iterator datam
    
    v_sql = """SELECT distinct
        to_char(his.MATCH_DATE,'yyyymm') iterator
    FROM 
        betting_mart.ml_poisson_features
    WHERE
        --2014_15 season dropped for stable features
        season <> '2014_2015'
        --2015_16 is first training season
        AND season <> '2015_2016'
        and season <> '2021_2022'
    ORDER BY 1;"""

    return dwh_conn.export_to_pandas(v_sql)

def get_training_data_home(month_filter):

    #load training data based on provided filter
    v_sql = """
    select
        round(home_goals_for_ema5,1)            home_goals_for_ema5,
        round(home_goals_against_ema5,1)        home_goals_against_ema5,
        round(away_goals_for_ema5,1)            away_goals_for_ema5,
        round(away_goals_against_ema5,1)        away_goals_against_ema5,
        round(home_xg_for_ema5,1)               home_xg_for_ema5,
        round(home_xg_against_ema5,1)           home_xg_against_ema5,
        round(away_xg_for_ema5,1)               away_xg_for_ema5,
        round(away_xg_against_ema5,1)           away_xg_against_ema5,
        round(home_shots_for_ema5,1)               home_shots_for_ema5,
        round(home_shots_against_ema5,1)           home_shots_against_ema5,
        round(away_shots_for_ema5,1)               away_shots_for_ema5,
        round(away_shots_against_ema5,1)           away_shots_against_ema5,
        round(home_sot_for_ema5,1)               home_sot_for_ema5,
        round(home_sot_against_ema5,1)           home_sot_against_ema5,
        round(away_sot_for_ema5,1)               away_sot_for_ema5,
        round(away_sot_against_ema5,1)           away_sot_against_ema5,
        round(home_corners_for_ema5,1)               home_corners_for_ema5,
        round(home_corners_against_ema5,1)           home_corners_against_ema5,
        round(away_corners_for_ema5,1)               away_corners_for_ema5,
        round(away_corners_against_ema5,1)           away_corners_against_ema5,
        round(home_deep_for_ema5,1)               home_deep_for_ema5,
        round(home_deep_against_ema5,1)           home_deep_against_ema5,
        round(away_deep_for_ema5,1)               away_deep_for_ema5,
        round(away_deep_against_ema5,1)           away_deep_against_ema5,
        round(home_ppda_for_ema5,1)               home_ppda_for_ema5,
        round(home_ppda_against_ema5,1)           home_ppda_against_ema5,
        round(away_ppda_for_ema5,1)               away_ppda_for_ema5,
        round(away_ppda_against_ema5,1)           away_ppda_against_ema5,
        full_time_home_goals               full_time_home_goals
    from
        betting_mart.ml_poisson_features
    where
        feature_type = 'EMA5 HA'
        and full_time_home_goals is not null
        and home_shots_for_ema5 is not null
        and away_shots_against_ema5 is not null
        and SEASON <> '2014_2015' and
        --filter for iterator
        and to_number(to_char(his.MATCH_DATE,'yyyymm')) < """ + str(month_filter) + ";"

    return dwh_conn.export_to_pandas(v_sql)


def get_training_data_away(month_filter):

    #load training data based on provided filter
    v_sql = """
    select
        round(home_goals_for_ema5,1)            home_goals_for_ema5,
        round(home_goals_against_ema5,1)        home_goals_against_ema5,
        round(away_goals_for_ema5,1)            away_goals_for_ema5,
        round(away_goals_against_ema5,1)        away_goals_against_ema5,
        round(home_xg_for_ema5,1)               home_xg_for_ema5,
        round(home_xg_against_ema5,1)           home_xg_against_ema5,
        round(away_xg_for_ema5,1)               away_xg_for_ema5,
        round(away_xg_against_ema5,1)           away_xg_against_ema5,
        round(home_shots_for_ema5,1)               home_shots_for_ema5,
        round(home_shots_against_ema5,1)           home_shots_against_ema5,
        round(away_shots_for_ema5,1)               away_shots_for_ema5,
        round(away_shots_against_ema5,1)           away_shots_against_ema5,
        round(home_sot_for_ema5,1)               home_sot_for_ema5,
        round(home_sot_against_ema5,1)           home_sot_against_ema5,
        round(away_sot_for_ema5,1)               away_sot_for_ema5,
        round(away_sot_against_ema5,1)           away_sot_against_ema5,
        round(home_corners_for_ema5,1)               home_corners_for_ema5,
        round(home_corners_against_ema5,1)           home_corners_against_ema5,
        round(away_corners_for_ema5,1)               away_corners_for_ema5,
        round(away_corners_against_ema5,1)           away_corners_against_ema5,
        round(home_deep_for_ema5,1)               home_deep_for_ema5,
        round(home_deep_against_ema5,1)           home_deep_against_ema5,
        round(away_deep_for_ema5,1)               away_deep_for_ema5,
        round(away_deep_against_ema5,1)           away_deep_against_ema5,
        round(home_ppda_for_ema5,1)               home_ppda_for_ema5,
        round(home_ppda_against_ema5,1)           home_ppda_against_ema5,
        round(away_ppda_for_ema5,1)               away_ppda_for_ema5,
        round(away_ppda_against_ema5,1)           away_ppda_against_ema5,
        full_time_away_goals               full_time_home_goals
    from
        betting_mart.ml_poisson_features
    where
        feature_type = 'EMA5 HA'
        and full_time_home_goals is not null
        and home_shots_for_ema5 is not null
        and away_shots_against_ema5 is not null
        and SEASON <> '2014_2015' and
        --filter for iterator
        and to_number(to_char(his.MATCH_DATE,'yyyymm')) < """ + str(month_filter) + ";"

    return dwh_conn.export_to_pandas(v_sql)




def get_pred_data(month_filter):

    #load training data based on provided filter
    v_sql = """
    select
        football_match_his_lid,
        --model features
        round(home_goals_for_ema5,1)            home_goals_for_ema5,
        round(home_goals_against_ema5,1)        home_goals_against_ema5,
        round(away_goals_for_ema5,1)            away_goals_for_ema5,
        round(away_goals_against_ema5,1)        away_goals_against_ema5,
        round(home_xg_for_ema5,1)               home_xg_for_ema5,
        round(home_xg_against_ema5,1)           home_xg_against_ema5,
        round(away_xg_for_ema5,1)               away_xg_for_ema5,
        round(away_xg_against_ema5,1)           away_xg_against_ema5,
        round(home_shots_for_ema5,1)               home_shots_for_ema5,
        round(home_shots_against_ema5,1)           home_shots_against_ema5,
        round(away_shots_for_ema5,1)               away_shots_for_ema5,
        round(away_shots_against_ema5,1)           away_shots_against_ema5,
        round(home_sot_for_ema5,1)               home_sot_for_ema5,
        round(home_sot_against_ema5,1)           home_sot_against_ema5,
        round(away_sot_for_ema5,1)               away_sot_for_ema5,
        round(away_sot_against_ema5,1)           away_sot_against_ema5,
        round(home_corners_for_ema5,1)               home_corners_for_ema5,
        round(home_corners_against_ema5,1)           home_corners_against_ema5,
        round(away_corners_for_ema5,1)               away_corners_for_ema5,
        round(away_corners_against_ema5,1)           away_corners_against_ema5,
        round(home_deep_for_ema5,1)               home_deep_for_ema5,
        round(home_deep_against_ema5,1)           home_deep_against_ema5,
        round(away_deep_for_ema5,1)               away_deep_for_ema5,
        round(away_deep_against_ema5,1)           away_deep_against_ema5,
        round(home_ppda_for_ema5,1)               home_ppda_for_ema5,
        round(home_ppda_against_ema5,1)           home_ppda_against_ema5,
        round(away_ppda_for_ema5,1)               away_ppda_for_ema5,
        round(away_ppda_against_ema5,1)           away_ppda_against_ema5
    from
        betting_mart.ml_poisson_features
    where
        feature_type = 'EMA5 HA'
        and full_time_home_goals is not null
        and home_shots_for_ema5 is not null
        and away_shots_against_ema5 is not null
        and SEASON <> '2014_2015' and
        --filter for iterator
        and to_number(to_char(his.MATCH_DATE,'yyyymm')) = """ + str(month_filter) + ";"


    return dwh_conn.export_to_pandas(v_sql)


## model simulation per month

In [3]:

#truncate result table
dwh_conn.execute('truncate table BETTING_DV.FOOTBALL_MATCH_HIS_L_S_ML_POISSON_EXPECTED_GOALS')

#get iterator data
df_iterator = get_iterator_data()

#
# Iterator per month
#

for index, row in df_iterator.iterrows():
    print('Prediction for: ' + str(row['ITERATOR']))
    
    
    #get data to train home expGoals model
    df_train_data = get_training_data_home(row['ITERATOR'])
    
    #split features and pred classes
    df_train_home_x = df_train_data.iloc[:,:24]
    df_train_home_y = df_train_data.iloc[:,24:25]
    
    
    #get data to train away expGoals model
    df_train_data = get_training_data_away(row['ITERATOR'])
    
    #split features and pred classes
    df_train_away_x = df_train_data.iloc[:,:24]
    df_train_away_y = df_train_data.iloc[:,24:25]
        
    
    #
    #train logistic regression models
    #
    
    print('...training logistic regression models')
    
    solvers = 'lbfgs'
    penalty = 'l2'
    c_values = 0.01
    
       
    lr_home_model = LogisticRegression(solver=solvers, penalty=penalty, C=c_values, random_state=0).fit(df_train_home_x, df_train_home_y)
    lr_away_model = LogisticRegression(solver=solvers, penalty=penalty, C=c_values, random_state=0).fit(df_train_away_x, df_train_away_y)

    
    #print('......training auc score: ' + str(roc_auc_score(df_train_y, xg_model.predict_proba(df_train_x),multi_class='ovo')))
    print('......home training score: ' + str(lr_model.score(df_train_home_x,df_train_home_y)))
    print('......away training score: ' + str(lr_model.score(df_train_away_x,df_train_away_y)))
    
    
    
    
    #
    # predicting logistic regression
    #
    
    #get data for prediction
    df_pred_data = get_pred_data(row['ITERATOR'])
    
    #do prediction
    df_pred_x = df_pred_data.iloc[:,1:25]
    
    print('...executing logistic regression prediction')  
    
    df_pred_home_expGoals = lr_home_model.predict_proba(df_pred_x)
    df_pred_away_expGoals = lr_away_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'Logistic Regression EMA5'
    df_result_data['home_exp_goals'] = df_pred_home_expGoals[:,0:1]
    df_result_data['away_exp_goals'] = df_pred_away_expGoals[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_ML_POISSON_EXPECTED_GOALS'))
    
    

ExaQueryError: 
(
    message     =>  object BETTING_DV.FOOTBALL_MATCH_HIS_L_S_ML_POISSON_EXPECTED_GOALS not found [line 1, column 16] (Session: 1735593120308920320)
    dsn         =>  ec2-18-198-93-169.eu-central-1.compute.amazonaws.com:8563
    user        =>  sys
    schema      =>  
    session_id  =>  1735593120308920320
    code        =>  42000
    query       =>  truncate table BETTING_DV.FOOTBALL_MATCH_HIS_L_S_ML_POISSON_EXPECTED_GOALS
)
